In [ ]:
import nibabel as nib
import numpy as np
from scipy.ndimage import distance_transform_edt
from skimage import measure

path_L = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\manual_segmentations\nii\Mandible_L.nii.gz"
path_R = r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\manual_segmentations\nii\Mandible_R.nii.gz"

img_L = nib.load(path_L)
img_R = nib.load(path_R)

seg_L = img_L.get_fdata() > 0
seg_R = img_R.get_fdata() > 0

affine = img_L.affine
spacing = np.abs(np.diag(affine))[:3]  

dist_L = distance_transform_edt(~seg_L, sampling=spacing)
dist_R = distance_transform_edt(~seg_R, sampling=spacing)

# equidistant field
D = dist_L - dist_R

mid_verts, mid_faces, _, _ = measure.marching_cubes(D, level=0)


def vox2mm(verts, affine):
    verts_h = np.c_[verts, np.ones(len(verts))]
    return (affine @ verts_h.T).T[:, :3]

midline_mm = vox2mm(mid_verts, affine)

print("Midline point cloud shape:", midline_mm.shape)
midline_mm[:10]  # preview first points


Midline point cloud shape: (141889, 3)


array([[  27.33520198,  204.56647426, -173.75      ],
       [  27.49031412,  204.09980041, -173.75      ],
       [  27.33520198,  204.09980041, -173.62919894],
       [  27.33520198,  203.12320042, -173.37531544],
       [  27.81593189,  203.12320042, -173.75      ],
       [  27.33520198,  202.14660043, -173.12114008],
       [  28.14138575,  202.14660043, -173.75      ],
       [  27.33520198,  201.17000043, -172.86666028],
       [  28.03409327,  201.17000043, -173.75      ],
       [  27.33520198,  200.19340044, -172.63552383]])

In [ ]:
import numpy as np
from scipy.signal import savgol_filter
import trimesh
pts = midline_mm.copy()

mean = pts.mean(axis=0)
X = pts - mean
U, S, Vt = np.linalg.svd(X, full_matrices=False)
principal_axis = Vt[0] 

t = X @ principal_axis  

nbins = 300 
bins = np.linspace(t.min(), t.max(), nbins+1)
bin_idx = np.digitize(t, bins) - 1

centroids = []
bin_centers = []
for i in range(nbins):
    sel = pts[bin_idx == i]
    if len(sel) > 0:
        centroids.append(sel.mean(axis=0))
        bin_centers.append(0.5*(bins[i]+bins[i+1]))

centroids = np.array(centroids)
if centroids.shape[0] < 5:
    raise RuntimeError("Too few bins with points; reduce nbins")

window = 11 if centroids.shape[0] > 11 else (centroids.shape[0]//2*2+1)
poly = 3 if window > 3 else 1
smoothed = np.vstack([
    savgol_filter(centroids[:,0], window, poly),
    savgol_filter(centroids[:,1], window, poly),
    savgol_filter(centroids[:,2], window, poly),
]).T

midline_curve = smoothed
print("Midline curve shape:", midline_curve.shape)

np.savetxt(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\asymmetry\midline.csv", midline_curve, delimiter=",", header="x,y,z", comments="")

path = trimesh.load_path(midline_curve)
path.export(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\asymmetry\midline.ply")
print("Saved midline.csv and midline.ply")


Midline curve shape: (300, 3)
Saved midline.csv and midline.ply


In [ ]:
#to visualise on 3D slicer
import slicer
import numpy as np

import csv
midline_curve = []
with open(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x7axQ0Q1\asymmetry\midline.csv") as f:
    reader = csv.reader(f)
    for row in reader:
        if len(row) < 3: 
            continue
        try:
            x, y, z = map(float, row[:3])
            midline_curve.append([x, y, z])
        except:
            continue
midline_curve = np.array(midline_curve)

curve_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsCurveNode", "MandibleMidline")

for pt in midline_curve:
    curve_node.AddControlPoint(pt.tolist())

slicer.app.processEvents()
curve_node.SetDisplayVisibility(True)
